# 08-LEARN-DataAnalysisML

In [1]:
import pandas as pd

In [2]:
from constrain.lab.transformation import ODtime, ng_to_nmol

This is the data after data processing in the previous notebook. 

In [3]:
input_for_ml = pd.read_csv('../notebooks/input_for_ml.csv')
input_for_ml = input_for_ml.set_index('Line Name')
#input_for_ml.to_csv('ml_test_csv')

Lets setup AutoML

In [4]:
import h2o
from h2o.automl import H2OAutoML

In [97]:
# If this doesnt work - install java. 
# Start the H2O cluster (locally)
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,5 hours 38 mins
H2O_cluster_timezone:,Europe/Copenhagen
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_lucaslevassor_r1p5zc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.097 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### Import out dataframe to h20 object

In [6]:
df_test = h2o.H2OFrame(pd.concat([input_for_ml], axis='columns'))
df_test

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


0,1,2,3,Amt_norm
1,2,5,1,0.972701
1,3,8,1,0.269401
1,3,8,2,0.202699
1,2,6,2,1.13327
1,1,5,3,0.98107
1,1,8,3,0.223689
1,3,7,3,0
1,2,8,5,0.255908
1,1,8,5,0
1,1,5,6,0


### RUN automl

In [7]:
# Select the columns we want to train on
feature_cols = [0, 1, 2, 3]

Set some arguments for the autoML

In [8]:
aml = H2OAutoML(
    max_runtime_secs=(3600 * 1),  # 1 hour
    max_models=None,  # no limit
    nfolds=5,         # number of folds for k-fold cross-validation (nfolds=0 disables cross-validation)
    seed=1            # Reproducibility
)

Train a model based on the target and the df

In [13]:
'''
lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
out_path = '../data/processed/ML_data_leaderboard'

for m_id in model_ids:
     mdl = h2o.get_model(m_id)
     h2o.save_model(model=mdl, path=out_path, force=True)
'''

"\nlb = aml.leaderboard\nmodel_ids = list(lb['model_id'].as_data_frame().iloc[:,0])\nout_path = '../data/processed/ML_data_leaderboard'\n\nfor m_id in model_ids:\n     mdl = h2o.get_model(m_id)\n     h2o.save_model(model=mdl, path=out_path, force=True)\n"

In [14]:
import os
#h2o.export_file(lb, os.path.join(out_path, 'aml_leaderboard.h2o'), force=True)


In [15]:
#models_path = "../data/processed/ML_data_leaderboard"

#lb = h2o.import_file(path=os.path.join(models_path, "aml_leaderboard.h2o"))

#lb.head(rows=10)

In [16]:
#se_all = h2o.load_model(os.path.join(models_path, "StackedEnsemble_BestOfFamily_4_AutoML_6_20220724_170618"))
# Get the Stacked Ensemble metalearner model
#metalearner = h2o.get_model(se_all.metalearner().model_id)

In [17]:
#%matplotlib inline
#metalearner.std_coef_plot(num_of_features=20)

In [18]:
#se_best_of_family = h2o.load_model(os.path.join(models_path, "StackedEnsemble_BestOfFamily_4_AutoML_6_20220724_170618"))
# Get the Stacked Ensemble metalearner model
#metalearner = h2o.get_model(se_best_of_family.metalearner()['name'])

#%matplotlib inline
#metalearner.std_coef_plot(num_of_features=10)

In [19]:
# Run ML with new input 

new_input_for_ml = pd.read_csv('../data/processed/input_for_ml_reseq.csv')
new_input_for_ml

,Line Name,0,1,2,3,Amt_norm
0,yp49_A01,1,2,5,1,0.972701
1,yp49_A02,1,3,8,1,0.269401
2,yp49_A04,1,3,8,2,0.202699
3,yp49_A06,1,2,6,2,1.133272
4,yp49_A07,1,1,5,3,0.981070
...,...,...,...,...,...,...
161,yp51_E05,8,3,6,8,0.000000
162,yp51_E07,8,1,8,9,6.795506
163,yp51_E08,8,4,5,9,0.668263
164,yp51_E11,8,3,7,10,0.000000


In [20]:
df_test = h2o.H2OFrame(pd.concat([new_input_for_ml], axis='columns'))
df_test.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Rows:166
Cols:6




,Line Name,0,1,2,3,Amt_norm
type,string,int,int,int,int,real
mins,NaN,1.0,1.0,5.0,1.0,0.0
mean,NaN,4.566265060240965,2.6927710843373482,6.566265060240965,5.319277108433736,17.20841465493667
maxs,NaN,8.0,4.0,8.0,10.0,277.3078163064027
sigma,NaN,2.2519635665513875,1.1581890914364779,1.1723353650321857,2.8392042347076565,40.8029369946811
zeros,0,0,0,0,0,59
missing,0,0,0,0,0,0
0,yp49_A01,1.0,2.0,5.0,1.0,0.972700677969159
1,yp49_A02,1.0,3.0,8.0,1.0,0.2694007038642038
2,yp49_A04,1.0,3.0,8.0,2.0,0.2026994926097329


In [21]:
# Since the data is categorical we need to make it from numerical to categorical 

df_test['0']= df_test['0'].asfactor()
df_test['1']= df_test['1'].asfactor()
df_test['2'] = df_test['2'].asfactor()
df_test['3'] = df_test['3'].asfactor()


In [22]:
df_test.describe()

Rows:166
Cols:6




,Line Name,0,1,2,3,Amt_norm
type,string,enum,enum,enum,enum,real
mins,NaN,,,,,0.0
mean,NaN,,,,,17.20841465493667
maxs,NaN,,,,,277.3078163064027
sigma,NaN,,,,,40.8029369946811
zeros,0,,,,,59
missing,0,0,0,0,0,0
0,yp49_A01,1,2,5,1,0.972700677969159
1,yp49_A02,1,3,8,1,0.2694007038642038
2,yp49_A04,1,3,8,2,0.2026994926097329


Set some arguments for the autoML. 


Important here is that we dont split the dataset but rather keep the cross-validation validate a model internally, i.e., estimate the model performance without having to sacrifice a validation split. See: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/cross-validation.html


In [30]:
# Select the columns we want to train on
feature_cols = [0, 1, 2, 3]

# Initialize H2O autoML class
aml = H2OAutoML(
    max_runtime_secs=int(3600 * 1),  # 1 hour, if unlimited time is wanted then set this to zero = 0
    max_models=None,  # no limit
    nfolds=5,         # number of folds for k-fold cross-validation (nfolds=0 disables cross-validation)
    seed=1,            # Reproducibility
    keep_cross_validation_predictions=True 
)

Train a model based on the target and the df

In [31]:
%%time

aml.train(
     x=feature_cols,
     y='Amt_norm',
     training_frame=df_test,
 )


AutoML progress: |
10:19:39.431: _train param, Dropping bad and constant columns: [Line Name]


10:19:39.995: _train param, Dropping bad and constant columns: [Line Name]


10:19:40.223: _train param, Dropping bad and constant columns: [Line Name]
10:19:40.223: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 166.0.
10:19:40.225: _train param, Dropping unused columns: [Line Name]
10:19:40.345: _train param, Dropping bad and constant columns: [Line Name]
10:19:40.568: _train param, Dropping bad and constant columns: [Line Name]

█
10:19:40.873: _train param, Dropping bad and constant columns: [Line Name]
10:19:41.218: _train param, Dropping bad and constant columns: [Line Name]

█
10:19:41.484: _train param, Dropping bad and constant columns: [Line Name]
10:19:41.692: _train param, Dropping unused columns: [Line Name]
10:19:41.814: _train param, Dropping unused columns: [Line Name]

█
10:19:41.954: _train

,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,19,Input,15.0,,,,,,,,,
1,,2,100,RectifierDropout,10.0,0.0,0.0,0.194622,0.369268,0.0,-0.023686,0.202101,0.490802,0.273122
2,,3,100,RectifierDropout,10.0,0.0,0.0,0.019142,0.039293,0.0,-0.009435,0.126848,0.984915,0.101737
3,,4,100,RectifierDropout,10.0,0.0,0.0,0.022009,0.068832,0.0,-0.015851,0.107741,1.010324,0.09082
4,,5,1,Linear,,0.0,0.0,0.000488,0.000534,0.0,0.01806,0.10248,0.346199,0.0




ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 273.4177422936487
RMSE: 16.535348266475935
MAE: 9.542883931243875
RMSLE: 1.4747267539097657
Mean Residual Deviance: 273.4177422936487

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 571.6042544804251
RMSE: 23.908246578961517
MAE: 11.580266277641334
RMSLE: NaN
Mean Residual Deviance: 571.6042544804251

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,11.563775,4.765719,14.301377,8.049455,11.953172,5.808622,17.706247
1,mean_residual_deviance,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
2,mse,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
3,r2,0.605487,0.245662,0.226033,0.814406,0.814654,0.654250,0.518090
4,residual_deviance,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
5,rmse,22.507841,8.934957,27.546583,17.866900,22.305645,10.740706,34.079370
6,rmsle,1.191783,0.386785,NaN,0.918285,1.465281,NaN,NaN



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
0,,2022-07-26 10:56:43,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN
1,,2022-07-26 10:56:43,50.940 sec,14188 obs/sec,10.0,1,1660.0,32.623156,1064.270300,22.676532,0.356878
2,,2022-07-26 10:56:48,55.987 sec,25088 obs/sec,780.0,78,129480.0,16.561103,274.270124,7.033766,0.834263
3,,2022-07-26 10:56:53,1 min 1.023 sec,30451 obs/sec,1870.0,187,310420.0,16.198574,262.393804,8.279574,0.841440
4,,2022-07-26 10:56:57,1 min 4.559 sec,33497 obs/sec,2770.0,277,459820.0,16.535348,273.417742,9.542884,0.834778



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,0.1,1.000000,1.000000,0.107484
1,0.8,0.816173,0.816173,0.087726
2,1.3,0.800856,0.800856,0.086079
3,2.7,0.760436,0.760436,0.081735
4,1.4,0.647474,0.647474,0.069593
5,0.2,0.620730,0.620730,0.066719
6,0.5,0.567350,0.567350,0.060981
7,1.2,0.546562,0.546562,0.058747
8,2.5,0.531041,0.531041,0.057078
9,0.6,0.507082,0.507082,0.054503


## Processing model


In [35]:
lb = aml.leaderboard
print(lb)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
DeepLearning_grid_3_AutoML_2_20220726_101939_model_1,23.9082,571.604,11.5803,nan,571.604
DeepLearning_grid_3_AutoML_2_20220726_101939_model_10,23.9759,574.846,13.5684,nan,574.846
StackedEnsemble_BestOfFamily_7_AutoML_2_20220726_101939,24.2198,586.599,12.1232,nan,586.599
StackedEnsemble_BestOfFamily_5_AutoML_2_20220726_101939,24.2989,590.436,12.0126,nan,590.436
DeepLearning_grid_2_AutoML_2_20220726_101939_model_8,24.3408,592.475,13.8162,nan,592.475
DeepLearning_grid_2_AutoML_2_20220726_101939_model_4,24.4395,597.288,13.9553,nan,597.288
DeepLearning_grid_1_AutoML_2_20220726_101939_model_74,24.5496,602.68,11.1824,nan,602.68
DeepLearning_grid_2_AutoML_2_20220726_101939_model_37,24.6449,607.37,11.4348,1.1355,607.37
DeepLearning_grid_1_AutoML_2_20220726_101939_model_236,24.8821,619.118,12.1355,nan,619.118
DeepLearning_grid_2_AutoML_2_20220726_101939_model_10,24.9309,621.548,13.8228,nan,621.548


In [53]:
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
len(model_ids)

377

Exporting the results

In [54]:
# saving the models
lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])

# get rid of models with stacked ensemble - they; the tree-based models, the tree ensemble (GBM or Random Forest) ends up memorizing the training data.
#The model has a poor time generalizing on validation data
for model in model_ids:
    if 'Stacked' in model:
        model_ids.remove(model)
        
len(model_ids)

365

## Lets save the model


In [55]:
# saving the models
lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])


# get rid of models with stacked ensemble - they; the tree-based models, the tree ensemble (GBM or Random Forest) ends up memorizing the training data.
#The model has a poor time generalizing on validation data
for model in model_ids:
    if 'StackedEnsemble' in model:
        model_ids.remove(model)

out_path = '../data/processed/ML_data_leaderboard2'

for m_id in model_ids:
     mdl = h2o.get_model(m_id)
     h2o.save_model(model=mdl, path=out_path, force=True)


In [56]:
len(model_ids)

365

In [57]:
# saving the leaderboard
import os
h2o.export_file(lb, os.path.join(out_path, 'aml_leaderboard.h2o'), force=True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [73]:
best_model = aml.get_best_model()


SyntaxError: invalid syntax (3847323714.py, line 1)

Importing them 

In [48]:
models_path = "../data/processed/ML_data_leaderboard2"

lb = h2o.import_file(path=os.path.join(models_path, "aml_leaderboard.h2o"))

lb.head(rows=1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
DeepLearning_grid_3_AutoML_2_20220726_101939_model_1,23.9082,571.604,11.5803,nan,571.604


Now we want to extract the best models that are not StackedEnsemble

In [59]:
lb = aml.leaderboard
df_from_h2o_object = lb.as_data_frame(use_pandas=True, header=True)
df_from_h2o_object

,model_id,rmse,mse,mae,rmsle,mean_residual_deviance
0,DeepLearning_grid_3_AutoML_2_20220726_101939_model_1,23.908247,571.604254,11.580266,NaN,571.604254
1,DeepLearning_grid_3_AutoML_2_20220726_101939_model_10,23.975938,574.845602,13.568386,NaN,574.845602
2,StackedEnsemble_BestOfFamily_7_AutoML_2_20220726_101939,24.219809,586.599165,12.123153,NaN,586.599165
3,StackedEnsemble_BestOfFamily_5_AutoML_2_20220726_101939,24.298898,590.436432,12.012639,NaN,590.436432
4,DeepLearning_grid_2_AutoML_2_20220726_101939_model_8,24.340811,592.475085,13.816188,NaN,592.475085
...,...,...,...,...,...,...
372,DeepLearning_1_AutoML_2_20220726_101939,36.374303,1323.089948,21.293621,NaN,1323.089948
373,XGBoost_grid_1_AutoML_2_20220726_101939_model_13,36.961483,1366.151248,17.438055,NaN,1366.151248
374,XGBoost_grid_1_AutoML_2_20220726_101939_model_8,37.131715,1378.764276,16.353482,1.347843,1378.764276
375,DeepLearning_grid_3_AutoML_2_20220726_101939_model_75,37.172718,1381.810943,18.404644,NaN,1381.810943


In [60]:

index_to_delete = []
for index, row in df_from_h2o_object.iterrows():
    if 'StackedEnsemble' in row['model_id']:
        index_to_delete.append(index)

data_wo_ensemble = df_from_h2o_object.drop(labels=index_to_delete, axis=0)
final_df = h2o.H2OFrame(data_wo_ensemble)
final_df

,model_id,rmse,mse,mae,rmsle,mean_residual_deviance
0,DeepLearning_grid_3_AutoML_2_20220726_101939_model_1,23.908247,571.604254,11.580266,NaN,571.604254
1,DeepLearning_grid_3_AutoML_2_20220726_101939_model_10,23.975938,574.845602,13.568386,NaN,574.845602
4,DeepLearning_grid_2_AutoML_2_20220726_101939_model_8,24.340811,592.475085,13.816188,NaN,592.475085
5,DeepLearning_grid_2_AutoML_2_20220726_101939_model_4,24.439478,597.288077,13.955344,NaN,597.288077
6,DeepLearning_grid_1_AutoML_2_20220726_101939_model_74,24.549551,602.680439,11.182408,NaN,602.680439
...,...,...,...,...,...,...
372,DeepLearning_1_AutoML_2_20220726_101939,36.374303,1323.089948,21.293621,NaN,1323.089948
373,XGBoost_grid_1_AutoML_2_20220726_101939_model_13,36.961483,1366.151248,17.438055,NaN,1366.151248
374,XGBoost_grid_1_AutoML_2_20220726_101939_model_8,37.131715,1378.764276,16.353482,1.347843,1378.764276
375,DeepLearning_grid_3_AutoML_2_20220726_101939_model_75,37.172718,1381.810943,18.404644,NaN,1381.810943


In [84]:
best_model = aml.get_best_model()

In [87]:
best_model.cross_validation_holdout_predictions()

predict
3.30254
2.48685
4.96711
-1.5141
-2.01412
-2.30634
1.68131
5.97876
2.50321
-2.01412


In [94]:
frame = h2o.get_frame('DeepLearning_grid_3_AutoML_2_20220726_101939_model_1')
print(frame)

None


In [95]:

se_all = h2o.load_model(os.path.join(models_path, str(best_model)))
se_all

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_3_AutoML_2_20220726_101939_model_1


Status of Neuron Layers: predicting Amt_norm, regression, gaussian distribution, Quadratic loss, 22,301 weights/biases, 269.1 KB, 459,820 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,19,Input,15.0,,,,,,,,,
1,,2,100,RectifierDropout,10.0,0.0,0.0,0.194622,0.369268,0.0,-0.023686,0.202101,0.490802,0.273122
2,,3,100,RectifierDropout,10.0,0.0,0.0,0.019142,0.039293,0.0,-0.009435,0.126848,0.984915,0.101737
3,,4,100,RectifierDropout,10.0,0.0,0.0,0.022009,0.068832,0.0,-0.015851,0.107741,1.010324,0.09082
4,,5,1,Linear,,0.0,0.0,0.000488,0.000534,0.0,0.01806,0.10248,0.346199,0.0




ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 273.4177422936487
RMSE: 16.535348266475935
MAE: 9.542883931243875
RMSLE: 1.4747267539097657
Mean Residual Deviance: 273.4177422936487

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 571.6042544804251
RMSE: 23.908246578961517
MAE: 11.580266277641334
RMSLE: NaN
Mean Residual Deviance: 571.6042544804251

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,11.563775,4.765719,14.301377,8.049455,11.953172,5.808622,17.706247
1,mean_residual_deviance,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
2,mse,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
3,r2,0.605487,0.245662,0.226033,0.814406,0.814654,0.654250,0.518090
4,residual_deviance,570.469670,406.275880,758.814300,319.226100,497.541800,115.362750,1161.403300
5,rmse,22.507841,8.934957,27.546583,17.866900,22.305645,10.740706,34.079370
6,rmsle,1.191783,0.386785,NaN,0.918285,1.465281,NaN,NaN



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
0,,2022-07-26 10:56:43,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN
1,,2022-07-26 10:56:43,50.940 sec,14188 obs/sec,10.0,1,1660.0,32.623156,1064.270300,22.676532,0.356878
2,,2022-07-26 10:56:48,55.987 sec,25088 obs/sec,780.0,78,129480.0,16.561103,274.270124,7.033766,0.834263
3,,2022-07-26 10:56:53,1 min 1.023 sec,30451 obs/sec,1870.0,187,310420.0,16.198574,262.393804,8.279574,0.841440
4,,2022-07-26 10:56:57,1 min 4.559 sec,33497 obs/sec,2770.0,277,459820.0,16.535348,273.417742,9.542884,0.834778



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,0.1,1.000000,1.000000,0.107484
1,0.8,0.816173,0.816173,0.087726
2,1.3,0.800856,0.800856,0.086079
3,2.7,0.760436,0.760436,0.081735
4,1.4,0.647474,0.647474,0.069593
5,0.2,0.620730,0.620730,0.066719
6,0.5,0.567350,0.567350,0.060981
7,1.2,0.546562,0.546562,0.058747
8,2.5,0.531041,0.531041,0.057078
9,0.6,0.507082,0.507082,0.054503


H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/Users/lucaslevassor/projects/ConStrain/notebooks/../data/processed/ML_data_leaderboard2/ msg: File not found
  Request: POST /99/Models.bin/
    data: {'dir': '../data/processed/ML_data_leaderboard2/'}


In [96]:
se_all

NameError: name 'se_all' is not defined

In [80]:
metalearner = h2o.get_model(se_all.metalearner().model_id)


NameError: name 'se_all' is not defined

In [ ]:
#se_all = h2o.load_model(os.path.join(models_path, "StackedEnsemble_BestOfFamily_4_AutoML_6_20220724_170618"))
# Get the Stacked Ensemble metalearner model
#metalearner = h2o.get_model(se_all.metalearner().model_id)